<a href="https://colab.research.google.com/github/sumukhj1219/pyspark/blob/main/Copy_of_Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
employees = [
    (1, "Alice", 25, "HR", 5000),
    (2, "Bob", 30, "IT", 6000),
    (3, "Charlie", 35, "IT", 7000),
    (4, "David", 28, "HR", 5500),
    (5, "Eva", 40, "Finance", 6500),
    (6, "Frank", 29, "Finance", 4800)
]

emp_cols = ["EmpID", "Name", "Age", "Department", "Salary"]

In [ ]:
departments = [
    ("HR", "New York"),
    ("IT", "San Francisco"),
    ("Finance", "Chicago")
]

dept_cols = ["Department", "Location"]

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("EmployeeDepatment").getOrCreate()

df_emp = spark.createDataFrame(employees, emp_cols)
df_dept = spark.createDataFrame(departments, dept_cols)

print(df_emp)
print(df_dept)

DataFrame[EmpID: bigint, Name: string, Age: bigint, Department: string, Salary: bigint]
DataFrame[Department: string, Location: string]


In [ ]:
df_joined = df_emp.join(
    df_dept,
    on="Department",
    how="inner"
)
df_result = df_joined.select(
    "EmpID",
    "Name",
    "Age",
    "Department",
    "Salary",
    "Location"
)
df_result.show()

+-----+-------+---+----------+------+-------------+
|EmpID|   Name|Age|Department|Salary|     Location|
+-----+-------+---+----------+------+-------------+
|    5|    Eva| 40|   Finance|  6500|      Chicago|
|    6|  Frank| 29|   Finance|  4800|      Chicago|
|    1|  Alice| 25|        HR|  5000|     New York|
|    4|  David| 28|        HR|  5500|     New York|
|    2|    Bob| 30|        IT|  6000|San Francisco|
|    3|Charlie| 35|        IT|  7000|San Francisco|
+-----+-------+---+----------+------+-------------+



In [ ]:
from pyspark.sql.functions import when, col

df_with_category = df_result.withColumn(
    "Salary_Category",
    when(col("Salary") >= 6500, "High")
    .when((col("Salary") >= 5000) & (col("Salary") <= 6499), "Medium")
    .otherwise("Low")
)

df_with_category.show()


+-----+-------+---+----------+------+-------------+---------------+
|EmpID|   Name|Age|Department|Salary|     Location|Salary_Category|
+-----+-------+---+----------+------+-------------+---------------+
|    5|    Eva| 40|   Finance|  6500|      Chicago|           High|
|    6|  Frank| 29|   Finance|  4800|      Chicago|            Low|
|    1|  Alice| 25|        HR|  5000|     New York|         Medium|
|    4|  David| 28|        HR|  5500|     New York|         Medium|
|    2|    Bob| 30|        IT|  6000|San Francisco|         Medium|
|    3|Charlie| 35|        IT|  7000|San Francisco|           High|
+-----+-------+---+----------+------+-------------+---------------+



In [ ]:
from pyspark.sql.functions import when, col

df_with_age_group = df_with_category.withColumn(
    "Age_Group",
    when(col("Age") < 30, "Young")
    .when((col("Age") >= 30) & (col("Age") <= 39), "Mid")
    .otherwise("Senior")
)

df_with_age_group.show()


+-----+-------+---+----------+------+-------------+---------------+---------+
|EmpID|   Name|Age|Department|Salary|     Location|Salary_Category|Age_Group|
+-----+-------+---+----------+------+-------------+---------------+---------+
|    5|    Eva| 40|   Finance|  6500|      Chicago|           High|   Senior|
|    6|  Frank| 29|   Finance|  4800|      Chicago|            Low|    Young|
|    1|  Alice| 25|        HR|  5000|     New York|         Medium|    Young|
|    4|  David| 28|        HR|  5500|     New York|         Medium|    Young|
|    2|    Bob| 30|        IT|  6000|San Francisco|         Medium|      Mid|
|    3|Charlie| 35|        IT|  7000|San Francisco|           High|      Mid|
+-----+-------+---+----------+------+-------------+---------------+---------+



In [ ]:
from pyspark.sql.functions import avg

avg_salary_df = df_joined.groupBy("Department") \
    .agg(avg("Salary").alias("Average_Salary"))

avg_salary_df.show()


+----------+--------------+
|Department|Average_Salary|
+----------+--------------+
|   Finance|        5650.0|
|        HR|        5250.0|
|        IT|        6500.0|
+----------+--------------+



In [ ]:
from pyspark.sql.functions import max

max_salary_df = df_joined.groupBy("Department") \
    .agg(max("Salary").alias("Highest_Salary"))

max_salary_df.show()

+----------+--------------+
|Department|Highest_Salary|
+----------+--------------+
|   Finance|          6500|
|        HR|          5500|
|        IT|          7000|
+----------+--------------+



In [ ]:
from pyspark.sql.functions import count

count_location_df = df_joined.groupBy("Location") \
    .agg(count("EmpID").alias("Employee_Count"))

count_location_df.show()


+-------------+--------------+
|     Location|Employee_Count|
+-------------+--------------+
|San Francisco|             2|
|      Chicago|             2|
|     New York|             2|
+-------------+--------------+



In [ ]:
top_3_df = df_joined.orderBy("Salary", ascending=False).limit(3)
top_3_df.show()

+----------+-----+-------+---+------+-------------+
|Department|EmpID|   Name|Age|Salary|     Location|
+----------+-----+-------+---+------+-------------+
|        IT|    3|Charlie| 35|  7000|San Francisco|
|   Finance|    5|    Eva| 40|  6500|      Chicago|
|        IT|    2|    Bob| 30|  6000|San Francisco|
+----------+-----+-------+---+------+-------------+



In [ ]:
older_than_30_df = df_joined \
    .filter(df_joined.Age > 30) \
    .orderBy("Salary", ascending=False)

older_than_30_df.show()


+----------+-----+-------+---+------+-------------+
|Department|EmpID|   Name|Age|Salary|     Location|
+----------+-----+-------+---+------+-------------+
|        IT|    3|Charlie| 35|  7000|San Francisco|
|   Finance|    5|    Eva| 40|  6500|      Chicago|
+----------+-----+-------+---+------+-------------+



In [ ]:
df_joined.createOrReplaceTempView("employees_with_dept")
spark.sql("""
    SELECT Department, AVG(Salary) AS avg_salary
    FROM employees_with_dept
    GROUP BY Department
    ORDER BY avg_salary DESC
    LIMIT 1
""").show()


+----------+----------+
|Department|avg_salary|
+----------+----------+
|        IT|    6500.0|
+----------+----------+



In [ ]:
spark.sql("""
    SELECT e.*
    FROM employees_with_dept e
    JOIN (
        SELECT Department, AVG(Salary) AS avg_salary
        FROM employees_with_dept
        GROUP BY Department
    ) d
    ON e.Department = d.Department
    WHERE e.Salary > d.avg_salary
""").show()


+----------+-----+-------+---+------+-------------+
|Department|EmpID|   Name|Age|Salary|     Location|
+----------+-----+-------+---+------+-------------+
|   Finance|    5|    Eva| 40|  6500|      Chicago|
|        HR|    4|  David| 28|  5500|     New York|
|        IT|    3|Charlie| 35|  7000|San Francisco|
+----------+-----+-------+---+------+-------------+



In [ ]:
spark.sql("""
    SELECT *
    FROM employees_with_dept
    WHERE Salary > (
        SELECT AVG(Salary)
        FROM employees_with_dept
    )
""").show()


+----------+-----+-------+---+------+-------------+
|Department|EmpID|   Name|Age|Salary|     Location|
+----------+-----+-------+---+------+-------------+
|   Finance|    5|    Eva| 40|  6500|      Chicago|
|        IT|    2|    Bob| 30|  6000|San Francisco|
|        IT|    3|Charlie| 35|  7000|San Francisco|
+----------+-----+-------+---+------+-------------+

